In [1]:
import numpy as np 
import pandas as pd 
import gc
import re
import os

train_file = open('train.ft.txt')
test_file = open('test.ft.txt')

train_file_lines = train_file.readlines()
test_file_lines = test_file.readlines()

del train_file, test_file

train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file_lines]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file_lines]

for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d','0',train_sentences[i])
                                                       
for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])
        
del train_file_lines, test_file_lines

gc.collect()

labels_test = train_labels[90000:100000]
texts_test = train_sentences[90000:100000]
labels = train_labels[0:90000]
texts = train_sentences[0:90000]

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 200  # We will cut reviews after 100 words
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)  # https://keras.io/preprocessing/text/
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)  # This turns strings into lists of integer indices.

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

onehot = np.zeros((data.shape[0], max_words))
for i in range(data.shape[0]):
    for j in range(maxlen):
        temp2 = data [i]
        if temp2[j] != 0:
            a = temp2[j]
            onehot[i][a] = 1
            
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score
from sklearn.metrics import confusion_matrix
from models import SupervisedDBNClassification


# Loading dataset
X, Y = onehot, labels

# Data scaling
X = (X).astype(np.float32)

# Splitting data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=2/9, random_state=0)
#X_train = X[1000:5000]
#X_test = X[0:1000]
#Y_train = Y[1000:5000]
#Y_test= Y[0:1000]
print (X_train.shape)
print (X_test.shape)
print (Y_train.shape)
print (Y_test.shape)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 111047 unique tokens.
Shape of data tensor: (90000, 200)
Shape of label tensor: (90000,)
(70000, 10000)
(20000, 10000)
(70000,)
(20000,)


In [2]:
# Training
classifier = SupervisedDBNClassification(hidden_layers_structure=[16, 16],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.1,
                                         n_epochs_rbm=10,
                                         n_iter_backprop=10,
                                         batch_size=128,
                                         activation_function='relu',
                                         dropout_p=0.05)
classifier.fit(X_train, Y_train)

# Test
Y_pred = classifier.predict(X_test)
print('Done.\nAccuracy: %f' % accuracy_score(Y_test, Y_pred))

[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 41.101682
>> Epoch 2 finished 	RBM Reconstruction error 43.160544
>> Epoch 3 finished 	RBM Reconstruction error 42.062706
>> Epoch 4 finished 	RBM Reconstruction error 42.091315
>> Epoch 5 finished 	RBM Reconstruction error 41.968697
>> Epoch 6 finished 	RBM Reconstruction error 42.258303
>> Epoch 7 finished 	RBM Reconstruction error 42.799464
>> Epoch 8 finished 	RBM Reconstruction error 43.442384
>> Epoch 9 finished 	RBM Reconstruction error 44.006892
>> Epoch 10 finished 	RBM Reconstruction error 44.811724
>> Epoch 1 finished 	RBM Reconstruction error 16.348600
>> Epoch 2 finished 	RBM Reconstruction error 12.094534
>> Epoch 3 finished 	RBM Reconstruction error 8.081513
>> Epoch 4 finished 	RBM Reconstruction error 7.413871
>> Epoch 5 finished 	RBM Reconstruction error 6.407162
>> Epoch 6 finished 	RBM Reconstruction error 6.202371
>> Epoch 7 finished 	RBM Reconstruction error 5.881662
>> Epoch 8 finished 	RBM 